# A Challenging Example

The following example demonstrates several challenges that are common in reading text data: text encoding issues, multiple column formats, joining data across multiple files, and reshaping and recoding values, among other problems will be explored.

We are going to read data from the National Science Foundation (NSF) Higher Education Research and Development (HERD) Expenditures survey.  The data are collected annually from every higher education institution that spends more that $150,000 for research and development in a given fiscal year.  The data contain expenditure data for multiple academic fields and by funding source.

NSF provides access to public-use microdata via its [website](https://www.nsf.gov/statistics/herd/pub_data.cfm).  The data are available in multiple formats, including MS Excel, SAS and SPSS datafiles, and CSV formats.  We are going to work with the CSV files to demonstrate approaches to solving common data wrangling problems using real-world data.

As usual, we begin by loading the necessary libraries.

In [1]:
%config InlineBackend.figure_format ='retina'
%matplotlib inline

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_rows", 10)

# Read a CSV File

I'm going to read the 2017 data directly from the NSF website.  Because there are a number of errors that can occur when reading data this way (file errors, HTTP errors, syntax errors), I'm going to use a try-except block to catch any errors and report them in an elegant way.  This will make it easier to read my output, while still letting me know that something went wrong.  The basic specification is a "try" that calls something, and one or more "except" blocks that inform the user about what went awry if there was an error.  The final block, the "else" block will execute if no error has occurred.

Let's download the 2017 CSV file and see what we get...

In [2]:
try:
    herd_2017 = pd.read_csv("https://www.nsf.gov/statistics/herd/data/csv/herd_2017.csv")
except Exception as e:
    print("Something is amiss!\n\n{}\n".format(str(e)))
else:
    herd_2017.info()

Something is amiss!

'utf-8' codec can't decode byte 0x96 in position 51: invalid start byte



# Character Encoding

Hmm.  What just happened above?  This particular file was chosen to demonstrate a thorny issue that all data scientists will eventually face when working with text files: decoding errors.  The read_csv() function in pandas defaults to reading utf-8 encoded text.  Utf-8 in one of several unicode formats that accomodate multiple international locales.  It is a fairly compact format that can be used in multiple languages using non-English characters and is popular in passing text around the web.  Unfortunately, NSF chose to use another format.

(As an aside, a nice little history of character encodings can be read [here](https://danielmiessler.com/study/encoding/).)

In a perfect world, the file would carry a header that tells us what kind of encoding was used.  For example, a standard email has a header that looks something like this:
~~~
Content-type: text/plain; charset="UTF-8"
~~~
That is not the case here.  Fortunately, in the U.S., there are a few formats that are more commonly used than others: ascii-us, latin_1, and utf-8.  Since I know it isn't the last one, I used a trial-and-error method to determine that the characters are latin_1.  Looking up the codec in the python documentation, I found the latin_1 ISO code and placed it in my statement.  (Sometimes, determining the encoding can be a real pain!  A full list of codecs can be found [here](https://docs.python.org/3/library/codecs.html#standard-encodings).)

That being done, let's take another stab at reading our file...

In [3]:
try:
    herd_2017 = pd.read_csv("https://www.nsf.gov/statistics/herd/data/csv/herd_2017.csv",
                            encoding="iso-8859-1")
except Exception as e:
    print("\nSomething is amiss!\n\n{}\n".format(str(e)))
else:
    herd_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238288 entries, 0 to 238287
Data columns (total 23 columns):
inst_id                      238288 non-null int64
year                         238288 non-null int64
ncses_inst_id                235602 non-null object
ipeds_unitid                 224782 non-null float64
hbcu_flag                    238288 non-null int64
med_sch_flag                 238288 non-null object
hhe_flag                     238288 non-null int64
toi_code                     238288 non-null int64
hdg_code                     238288 non-null int64
toc_code                     238288 non-null int64
inst_name_long               238288 non-null object
inst_city                    238288 non-null object
inst_state_code              238288 non-null object
inst_zip                     238288 non-null object
questionnaire_no             238288 non-null object
question                     238288 non-null object
row                          238288 non-null object
column     

Success! We loaded 238,036 rows and 23 columns of data.  A quick look that the formats shows us a collection of numeric fields (int64, float64) and strings (object).  Let's take a quick look at the first 5 rows to get a feel for the contents:

In [4]:
herd_2017.head()

,inst_id,year,ncses_inst_id,ipeds_unitid,hbcu_flag,med_sch_flag,hhe_flag,toi_code,hdg_code,toc_code,...,inst_zip,questionnaire_no,question,row,column,data,status,othinfo,othinfo_s,standardized_agency_names
0,166,2017,U0626001,111966.0,0,T,0,1,2,2,...,90059,01.1,Inclusion of Institution funds,Competitively awarded internal grants,NaN,1.0,NaN,NaN,NaN,NaN
1,166,2017,U0626001,111966.0,0,T,0,1,2,2,...,90059,01.1,Inclusion of Institution funds,Other departmental funds,NaN,1.0,NaN,NaN,NaN,NaN
2,166,2017,U0626001,111966.0,0,T,0,1,2,2,...,90059,01.1,Inclusion of Institution funds,"Startup packages, bridge funding, seed funding",NaN,1.0,NaN,NaN,NaN,NaN
3,166,2017,U0626001,111966.0,0,T,0,1,2,2,...,90059,01.1,Inclusion of Institution funds,Tuition assistance for student research personnel,NaN,0.0,NaN,No expenditures,NaN,NaN
4,166,2017,U0626001,111966.0,0,T,0,1,2,2,...,90059,01.a,Source,Federal government,NaN,14461.0,NaN,NaN,NaN,NaN


# Data Typing

Looks like it loaded properly and did a fair job in picking the datatypes.  I can see three "flag" variables: *hbcu_flag*, *med_sch_flag*, and *hhe_flag*, which the [codebook](https://www.nsf.gov/statistics/herd/data/fy-2017-herd-dug-text-file-format.pdf) tells me are the yes/no indicators for Historically Black College and University (HBCU), institutions with a medical school, and High Hispanic Enrollment (HHE) institutions.  Two of the flags appear to be coded as (0, 1) values, while the medical school flag is in its historical T/F format.  For consistency, it is desirable to have these in the same format.  Since the 64-bit integer (int64) and object formats consume a lot of space for fairly primitive data (a boolean -- or bit -- is about as basic as they get!), I'd like to convert all three to boolean eventually.

Similarly, year is a four-digit integer, and my codes (toc_code, toi_code, hdg_code) are single-digit integers, so storing in 64-bit integer fields seems a little excessive.  For the unitid column, it is being converted to a floating-point variable, even though it is an integer value.  This is a quirk of python: integers cannot have null values, but the floating point types contain a NaN or "not a number" value to represent missing values.   I could let this go, but I'm going to convert this to a string.

All of these conversions can be handled after the load, but the read_csv() function gives us a handy "dtype" option that we can use to handle most (but not all) of the problems at the point of loading.  The _dtype_ statement can accept a single data type (like object) or a python dictionary with field names and their mapped types.  The first option is often handy when you've got problem values and want to load everything as text to search for the problem(s).  I'm going to use the second one to make the dataframe a little more compact and simplify my post-loading cleaning tasks a bit.

I've changed the year to a 16-bit integer (int16) and the single-digit codes to 8-bit (int8).  I've also changed the _ipeds_unitid_ column to string (object), and made the HBCU and HHE flags booleans (bool).  The _data_ field contains the reported values: small integers for some items and expenditures -- which can get quite large -- but are reported as whole numbers in the thousands.  The field can contain missing values, but doesn't need the precision of a 64-bit floating point number, so I've backed it down to a more reasonable (but still quite large) 32-bit float (float32).

So let's load this thing one more time, but with better data typing for our columns...

In [5]:
try:
    herd_2017 = pd.read_csv("https://www.nsf.gov/statistics/herd/data/csv/herd_2017.csv",
                            dtype =  {"inst_id":  np.int32,
                                      "year":  np.int16,
                                      "ncses_inst_id":  object,
                                      "ipeds_unitid":  object,
                                      "hbcu_flag":  bool,
                                      "med_sch_flag":  object,
                                      "hhe_flag":  bool,
                                      "toi_code":  np.int8,
                                      "hdg_code":  np.int8,
                                      "toc_code":  np.int8,
                                      "inst_name_long":  object,
                                      "inst_city":  object,
                                      "inst_state_code":  object,
                                      "inst_zip":  object,
                                      "questionnaire_no":  object,
                                      "question":  object,
                                      "row":  object,
                                      "column":  object,
                                      "data":  np.float32,
                                      "status":  object,
                                      "othinfo":  object,
                                      "othinfo_s":  object,
                                      "standardized_agency_names":  object},
                            encoding="iso-8859-1")
except Exception as e:
    print("\nSomething is amiss!\n\n{}\n".format(str(e)))
else:
    herd_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238288 entries, 0 to 238287
Data columns (total 23 columns):
inst_id                      238288 non-null int32
year                         238288 non-null int16
ncses_inst_id                235602 non-null object
ipeds_unitid                 224782 non-null object
hbcu_flag                    238288 non-null bool
med_sch_flag                 238288 non-null object
hhe_flag                     238288 non-null bool
toi_code                     238288 non-null int8
hdg_code                     238288 non-null int8
toc_code                     238288 non-null int8
inst_name_long               238288 non-null object
inst_city                    238288 non-null object
inst_state_code              238288 non-null object
inst_zip                     238288 non-null object
questionnaire_no             238288 non-null object
question                     238288 non-null object
row                          238288 non-null object
column           

Boom!  Note that our dataframe has gone from 41.8MB memory usage originally, down to 30.6MB.  This will become critical once we add additional years.  For completeness, and before we load all that additional data, let's take a look a the last five records of our dataframe to see if anything looks wacky at the end...

In [6]:
herd_2017.tail()

,inst_id,year,ncses_inst_id,ipeds_unitid,hbcu_flag,med_sch_flag,hhe_flag,toi_code,hdg_code,toc_code,...,inst_zip,questionnaire_no,question,row,column,data,status,othinfo,othinfo_s,standardized_agency_names
238283,353086,2017,NaN,NaN,False,T,False,1,1,2,...,37235,14K,Capitalized equipment expenditures by field an...,All,Nonfederal,1120.0,NaN,NaN,NaN,NaN
238284,353086,2017,NaN,NaN,False,T,False,1,1,2,...,37235,14K,Capitalized equipment expenditures by field an...,All,Total,10678.0,NaN,NaN,NaN,NaN
238285,353086,2017,NaN,NaN,False,T,False,1,1,2,...,37235,15,Personnel,Other personnel,NaN,5549.0,i,NaN,NaN,NaN
238286,353086,2017,NaN,NaN,False,T,False,1,1,2,...,37235,15,Personnel,Principal investigators,NaN,1054.0,i,NaN,NaN,NaN
238287,353086,2017,NaN,NaN,False,T,False,1,1,2,...,37235,15,Personnel,Total,NaN,6603.0,i,NaN,NaN,NaN


# Merging Multiple Data Objects

The values above look reasonable, so I'm feeling pretty confident about my code for loading a datafile.  I've still got to convert the medical school flag, but I'm going to hold the post-processing until after I've loaded all of the data.  Which brings me to my next task: combining multiple years' worth of data.

NSF has kindly included a year field to indicate the fiscal year of the data.  This is nice in that we don't have to do data gymnastics to differentiate the rows we'll be reading in for each institution across HERD surveys.  In looking at the documentation for the survey, I know that the present form of the survey began in FY2016, but is compatible with the data back to FY2010.  Prior to FY2010, they used a different survey, in addition to defining certain institutions differently.  Given that nest of potential headaches, we will limit our present task to working with the series from 2010 to present.

As of this writing, the most recent data are for fiscal year 2018.  I _could_ write a function that starts with 2010 and reads forward until it gets an error, but that is a bit of overkill for a one-time analysis.  I'm going to create an np.arange() range object that will cycle through all the available (and compatible) years.

Next, we need to determine if there is a reusable pattern in the URL's.  If not, we'll need to hardcode them in a python list.  In inspecting the URL's for the years in quesiton, I've determined that the only thing different in each is the year value in the filename:

~~~
https://www.nsf.gov/statistics/herd/data/csv/herd_2010.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2011.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2012.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2013.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2014.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2015.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2016.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2017.csv
https://www.nsf.gov/statistics/herd/data/csv/herd_2018.csv
~~~

Whew!  This makes the job __much__ easier!  Python's string functions make this type of operation a snap: I'll just loop through my year range, and concatenate the stub, the year, and the extension into a new URL each pass.

Next, we have to join -- or concatentate -- each year's dataframe into a single dataframe.  (This is also known as a _union_).  Pandas offeres a handy function called concat() that will do this trick for us.  We simply need to pass the current frame and the new frame in a list (using square brackets [ ]). To ensure that this works, I've created an empty dataframe named _raw_ prior to the loop.  Then, I loop through each year, creating the appropriate URL, reading the new data with proper specification, and then concatenate it with what is already being held in my _raw_ dataframe.  Note that I've added a _break_ statement in my _except_ block: I want it to stop processing if it encounters an error.  Easy peasy!

Let's give it a whirl...

In [7]:
first_year = 2010
last_year = 2018

raw = DataFrame()

for year in np.arange(first_year, last_year + 1):
    spec = 'https://www.nsf.gov/statistics/herd/data/csv/herd_' + str(year) + '.csv'
    try:
        raw = pd.concat([raw,
                     pd.read_csv(spec,
                                 dtype =  {"inst_id":  np.int32,
                                           "year":  np.int16,
                                           "ncses_inst_id":  object,
                                           "ipeds_unitid":  object,
                                           "hbcu_flag":  bool,
                                           "med_sch_flag":  object,
                                           "hhe_flag":  bool,
                                           "toi_code":  np.int8,
                                           "hdg_code":  np.int8,
                                           "toc_code":  np.int8,
                                           "inst_name_long":  object,
                                           "inst_city":  object,
                                           "inst_state_code":  object,
                                           "inst_zip":  object,
                                           "questionnaire_no":  object,
                                           "question":  object,
                                           "row":  object,
                                           "column":  object,
                                           "data":  np.float32,
                                           "status":  object,
                                           "othinfo":  object,
                                           "othinfo_s":  object,
                                           "standardized_agency_names":  object},
                                 encoding='iso-8859-1')])
    except Exception as e:
        print("\nSomething is amiss!\n\n{}\n".format(str(e)))
        break

raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2115686 entries, 0 to 238521
Data columns (total 23 columns):
inst_id                      int32
year                         int16
ncses_inst_id                object
ipeds_unitid                 object
hbcu_flag                    bool
med_sch_flag                 object
hhe_flag                     bool
toi_code                     int8
hdg_code                     int8
toc_code                     int8
inst_name_long               object
inst_city                    object
inst_state_code              object
inst_zip                     object
questionnaire_no             object
question                     object
row                          object
column                       object
data                         float32
status                       object
othinfo                      object
othinfo_s                    object
standardized_agency_names    object
dtypes: bool(2), float32(1), int16(1), int32(1), int8(3), object(15)
me

We just read more than 2 million rows of data from seven files without downloading a single physical file.  If you're as old as I am, you probably have vivid memories of downloading multiple files and munging them together.  Probably using a spreadsheet.  In a couple of dozen lines of code, we've done it all in one pass.  Now to tidy up the data so we can do some __real__ work!

# Conditional Calculations

Remember that medical school flag?  It really bothers me that it isn't in the same bool format as the other flags.  So, let's do something about that now.  (I promised we would!)

Our data are stored in something called a DataFrame, which is provided by the Pandas library.  Under the hood, each column in a dataframe is what is known as a Series.  The series object is great from an efficiency standpoint -- it speeds calculations (in fact, the functions for doing aggregations are part of the series object) and permits vectorized operations, but carries a price when it comes to doing conditional calculations.

Consider the following code:

~~~
if raw["med_sch_flag"] == "T":
    raw["med_sch_flag"] = True
else:
    raw["med_sch_flag"] = False
~~~

This seems to make logical sense, doesn't it?  Turns out, it is not so simple.  If you ran the code above, Python would hand back an error.  Specifically, _this_ error:

~~~
ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
~~~

So what does this mean?  Because the series object is built for vectorized operations, passing it to a boolean statement tells Python to try and test the entire series as a value.  So that first if statement is seen by Python and testing whether the Series itself was equal to "T" rather than each item in the series tested to equal "T".

Fortunately, the series object has some specialized functions to perform this kind of testing.  It just requires code that looks a little...different.  What I really want to do is test the value of the medical school flag for several possible values.  I've seen everything from T/F to Y/N used as boolean flags.  Instead of testing for each one, I'd like to test all of them simultaneously.  Enter the asin() function.  When we pass a list to the asin() function of a given series, it will hand back a True value for every matching row (and, conversely, a False value for every row value that is __not__ in the list.)

In [8]:
raw["med_sch_flag"] = raw["med_sch_flag"].isin(["T","TRUE", "True", "true", "t", "Y", "Yes", "1"])

raw.head()

,inst_id,year,ncses_inst_id,ipeds_unitid,hbcu_flag,med_sch_flag,hhe_flag,toi_code,hdg_code,toc_code,...,inst_zip,questionnaire_no,question,row,column,data,status,othinfo,othinfo_s,standardized_agency_names
0,166,2010,U0626001,111966,False,True,True,1,2,2,...,90059,01.a,Source,Federal government,NaN,20430.0,NaN,NaN,NaN,NaN
1,166,2010,U0626001,111966,False,True,True,1,2,2,...,90059,01.e,Source,Institution funds,NaN,1722.0,NaN,NaN,NaN,NaN
2,166,2010,U0626001,111966,False,True,True,1,2,2,...,90059,01.g,Source,Total,NaN,22152.0,NaN,NaN,NaN,NaN
3,166,2010,U0626001,111966,False,True,True,1,2,2,...,90059,03,Externally financed,"Grants, reimbursements, and other agreements",NaN,20430.0,NaN,NaN,NaN,NaN
4,166,2010,U0626001,111966,False,True,True,1,2,2,...,90059,03,Externally financed,Total,NaN,20430.0,NaN,NaN,NaN,NaN


In the output above, notice that we've successfully converted the medical school flag to a bool value like the other flags.  Note, that in all cases, we've used a lazy conversion method.  While we've tested for a True value, we've treated all non-True cases as False.  Strictly speaking, this is incomplete as an empty value does not mean the absence of something, but merely the absence of information.  In this case, I'd treat them as a _practical_ False value, but you should be aware that this is being done.

# Recoding Using map()

If we want to convert the codes in the _toi_code_, _hdg_code_, and _toc_code_ fields using __if__ statements, we'd run afoul of the same problem.  Instead, we use another handy function: map().  To use map, we need to create a dictionary containing the codes and corresponding labels.  Then, we map those labels to the values in the rows of the series.  I've gone to the codebook again and found that _toi_code_ contains the type of institution (only academic institutions are included in this datafile), the _hdg_code_ contains the highest degree offered, and the _toc_code_ contains the type of institution.  I've created dictionaries for all three and can now map those values on to the original fields, replacing the integer values with actual labels.  To save a little typing, I've wrapped my map() function into a function.

In [9]:
def item_recode(col, codings):
    answer = col.map(codings, na_action = "ignore")    
    return(answer)

toi = {1: "Academic"}

hdg = {1: "Doctorate",
       2: "Master’s",
       3: "Bachelor’s",
       4: "Associate’s",
       5: "No degree",
       6: "Professional degree"}

toc = {1: "Public",
       2: "Private"}

raw["toi_code"] = item_recode(raw["toi_code"], toi)
raw["hdg_code"] = item_recode(raw["hdg_code"], hdg)
raw["toc_code"] = item_recode(raw["toc_code"], toc)

raw.head()

,inst_id,year,ncses_inst_id,ipeds_unitid,hbcu_flag,med_sch_flag,hhe_flag,toi_code,hdg_code,toc_code,...,inst_zip,questionnaire_no,question,row,column,data,status,othinfo,othinfo_s,standardized_agency_names
0,166,2010,U0626001,111966,False,True,True,Academic,Master’s,Private,...,90059,01.a,Source,Federal government,NaN,20430.0,NaN,NaN,NaN,NaN
1,166,2010,U0626001,111966,False,True,True,Academic,Master’s,Private,...,90059,01.e,Source,Institution funds,NaN,1722.0,NaN,NaN,NaN,NaN
2,166,2010,U0626001,111966,False,True,True,Academic,Master’s,Private,...,90059,01.g,Source,Total,NaN,22152.0,NaN,NaN,NaN,NaN
3,166,2010,U0626001,111966,False,True,True,Academic,Master’s,Private,...,90059,03,Externally financed,"Grants, reimbursements, and other agreements",NaN,20430.0,NaN,NaN,NaN,NaN
4,166,2010,U0626001,111966,False,True,True,Academic,Master’s,Private,...,90059,03,Externally financed,Total,NaN,20430.0,NaN,NaN,NaN,NaN


Notice how the toi, hdg, and toc fields now have user-friendly labels.  It was a bit of work to get there, but we've now got the beginnings of a usable data frame.

# Adding Peer Groups

The next section adds boolean indicators for UNL's peers.  I've used the same isin() method of the pandas Series object to assign the value based on lists of unitid's.

In [ ]:
big_10 = [145637, 147767, 151351, 153658, 163286, 170976, 171100,
          174066, 181464, 186380, 204796, 214777, 240444, 243780]

regents = [181464, 153658, 153603, 126614,
           126818, 145637, 155317, 174066,
           178396, 204796, 243780]

raw["regents_peer"] = raw.ipeds_unitid.isin(regents)
raw["big_ten_peer"] = raw.ipeds_unitid.isin(big_10)

# Dropping Duplicates

So, one of the things we probably could use is a listing of participating institutions.  Since most HERD participants have submitted each year in our sample, we'll need to collapse the data frame to get an unduplicated table of institutions.  To do this, we use the drop_duplicates() function for a data frame.  As the name suggests, this method looks at the values of each column and eliminates any rows that contain duplicated values.

Obviously, the value and year columns will contain unique values for each year, so we will need to narrow our data frame to contain only institutional descriptors.  We'll do this by passing a list of the desired column names and unduplicate this narrower data frame.  The drop_duplicates() method takes an argument to determine which row to choose among a set of duplicates.  As a first pass, I've decided to keep only the first record and I've assigned this simplified data frame to an object (cleverly) named _institutions_.

In [10]:
keepers = ["inst_id", "ncses_inst_id", "ipeds_unitid", "inst_name_long",
           "inst_city", "inst_state_code", "inst_zip", "toc_code",
           "toi_code", "hdg_code", "hbcu_flag", "med_sch_flag", "hhe_flag"]

institutions = raw[keepers].drop_duplicates(keep="first")

institutions

,inst_id,ncses_inst_id,ipeds_unitid,inst_name_long,inst_city,inst_state_code,inst_zip,toc_code,toi_code,hdg_code,hbcu_flag,med_sch_flag,hhe_flag
0,166,U0626001,111966,Charles R. Drew University of Medicine and Sci...,Los Angeles,CA,90059,Private,Academic,Master’s,False,True,True
94,1002,U0047001,100654,Alabama Agricultural and Mechanical University,Normal,AL,35762,Public,Academic,Doctorate,True,False,False
364,1005,U0049001,100724,Alabama State University,Montgomery,AL,36101,Public,Academic,Doctorate,True,False,False
415,1009,U0323002,100858,Auburn University main campus,Auburn,AL,36849,Public,Academic,Doctorate,False,False,False
1021,1020,U1533001,101480,Jacksonville State University,Jacksonville,AL,36265,Public,Academic,Master’s,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229676,102015,NaN,485403,High Tech High Graduate School of Education,San Diego,CA,92106,Private,Academic,Master’s,False,False,False
230024,102034,U3107001,483036,Texas A&M University-Central Texas,Killeen,TX,76549,Public,Academic,Master’s,False,False,False
230172,102037,S3297001,141963,"University of Hawaii, system office",Honolulu,HI,96822,Public,Academic,Doctorate,False,False,False
230280,102040,S3355001,181747,University of Nebraska central administration,Lincoln,NE,68583,Public,Academic,Doctorate,False,False,False


This is a rather lazy way to create a data frame, however.  If anything changes for an institution over time (highest degree offered, perhaps), the drop_duplicates() method will return a row for each highest degree status.  Since institutions can and have had changes of this sort, we need a more robust method for finding a unique list.

We are going to use our year variable to make this determination.  If we use a groupby statement to aggregate by _inst_id_ (which does NOT change over time), we will be able to summarize year to obtain the _max_ value.  Using this value, we will obtain the most recent record for each institution.  This method is known as _last one in wins_ for obvious reasons!

In [18]:
# re-create keepers list to include year
keepers = ["inst_id", 'year', "ncses_inst_id", "ipeds_unitid",
           "inst_name_long", "inst_city", "inst_state_code", "inst_zip", "toc_code",
           "toi_code", "hdg_code", "hbcu_flag", "med_sch_flag", "hhe_flag"]

# de-duplicate items
year_max = raw[['inst_id', 'year']].groupby('inst_id').max()
year_max = year_max.reset_index()

# last one in wins
institutions = pd.merge(left = year_max,
                        right = raw[keepers],
                        on = ['inst_id', 'year'],
                        how = 'inner').drop_duplicates()

institutions

,inst_id,year,ncses_inst_id,ipeds_unitid,inst_name_long,inst_city,inst_state_code,inst_zip,toc_code,toi_code,hdg_code,hbcu_flag,med_sch_flag,hhe_flag
0,166,2018,U0626001,111966,Charles R. Drew University of Medicine and Sci...,Los Angeles,CA,90059,Private,Academic,Master’s,False,True,True
100,1002,2018,U0047001,100654,Alabama A&M University,Normal,AL,35762,Public,Academic,Doctorate,True,False,False
675,1005,2018,U0049001,100724,Alabama State University,Montgomery,AL,36104,Public,Academic,Doctorate,True,False,False
758,1009,2018,U0323002,100858,"Auburn University, Auburn",Auburn,AL,36849,Public,Academic,Doctorate,False,False,False
1466,1016,2011,U3365001,101879,University of North Alabama,Florence,AL,35632,Public,Academic,Master’s,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286892,330049,2013,N2840001,NaN,Skidaway Institute of Oceanography,Savannah,GA,31411,Public,Academic,Doctorate,False,False,False
286999,330050,2018,S0686018,190035,City University of New York system office,New York,NY,10017,Public,Academic,Doctorate,False,False,False
287172,330051,2018,S3338001,166665,University of Massachusetts central office,Boston,MA,02108,Public,Academic,Doctorate,False,False,False
287405,353086,2018,NaN,NaN,Vanderbilt University and Vanderbilt Universit...,Nashville,TN,37235,Private,Academic,Doctorate,False,True,False
